In [57]:
import yfinance as yf
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime as dt

In [50]:
tsla = yf.download('TSLA', dt.date.today() - dt.timedelta(1000), dt.date.today())
tsla = tsla.reset_index()

[*********************100%%**********************]  1 of 1 completed


In [51]:
c2c = np.divide(tsla['Close'].values[1:], tsla['Close'].values[:-1]) - 1
tsla['C2C'] = [0] + list(c2c)
tsla['TotRet'] = np.cumprod([1 + q for q in tsla['C2C']])


In [52]:
tsla

,Date,Open,High,Low,Close,Adj Close,Volume,C2C,TotRet
0,2021-07-19,209.963333,215.733337,207.096664,215.406662,215.406662,63891300,0.000000,1.000000
1,2021-07-20,217.330002,220.796661,213.500000,220.166672,220.166672,46461300,0.022098,1.022098
2,2021-07-21,219.869995,221.619995,216.763336,218.429993,218.429993,41859900,-0.007888,1.014035
3,2021-07-22,218.813339,220.723328,214.866669,216.419998,216.419998,45317100,-0.009202,1.004704
4,2021-07-23,215.453339,216.266663,212.433334,214.460007,214.460007,43814700,-0.009056,0.995605
...,...,...,...,...,...,...,...,...,...
683,2024-04-05,169.080002,170.860001,160.509995,164.899994,164.899994,141250700,-0.036292,0.765529
684,2024-04-08,169.339996,174.500000,167.789993,172.979996,172.979996,104423300,0.048999,0.803039
685,2024-04-09,172.910004,179.220001,171.919998,176.880005,176.880005,103232700,0.022546,0.821145
686,2024-04-10,173.039993,174.929993,170.009995,171.759995,171.759995,84532400,-0.028946,0.797375


In [74]:
win_size = 25
look_forward = 15
target_ret = 0.03

tsla_win = pd.DataFrame(columns=range(win_size))
target_vec = pd.DataFrame(columns=['Did I Hit?'])

for i in range(win_size, len(tsla.index) - look_forward):
    row = tsla.iloc[i - win_size:i]['C2C'].values
    tsla_win.loc[len(tsla_win.index)] = row
    if max(np.cumprod([1 + q for q in tsla.iloc[i:i+look_forward]['C2C']])) - 1 > target_ret:
        target_vec.loc[len(target_vec.index)] = 1
    else:
        target_vec.loc[len(target_vec.index)] = 0
    
train_tsla = tsla_win.sample(frac=0.7, random_state=0)
test_tsla = tsla_win.drop(train_tsla.index)

train_targ = target_vec.loc[train_tsla.index]
test_targ = target_vec.drop(train_targ.index)


In [71]:
train_tsla

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
531,0.000964,0.076878,-0.001089,0.004593,-0.050643,0.046855,-0.017817,-0.001707,-0.011889,0.100925,...,0.004599,-0.014672,-0.026239,-0.042315,0.008616,-0.011620,-0.014829,0.024449,0.015586,0.005515
493,0.009506,-0.021028,-0.007630,-0.017564,0.000668,0.008154,0.021729,0.012523,0.031985,0.010194,...,-0.003506,-0.032684,0.041962,0.003716,-0.023782,-0.026660,0.020503,-0.021055,-0.009493,-0.006960
306,0.029043,-0.034598,-0.011129,-0.063243,-0.000493,-0.028974,0.003418,0.020622,-0.075456,0.070052,...,0.052876,0.009981,0.002003,0.015238,-0.004289,0.001231,-0.056360,0.001535,-0.036413,-0.050080
624,-0.004341,-0.028725,-0.036661,0.004660,-0.019826,-0.017026,0.001463,-0.015976,0.001628,-0.006264,...,0.008383,-0.005030,-0.036454,0.022313,0.013398,0.010555,0.021154,-0.028104,-0.021847,0.025486
479,0.035544,-0.007421,-0.003466,0.018132,0.053389,-0.054618,0.019849,-0.030271,-0.060600,0.038000,...,-0.021028,-0.007630,-0.017564,0.000668,0.008154,0.021729,0.012523,0.031985,0.010194,-0.007091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,0.017212,-0.068101,-0.011036,-0.086145,0.029043,-0.034598,-0.011129,-0.063243,-0.000493,-0.028974,...,0.008402,-0.066474,0.034543,-0.014876,0.052876,0.009981,0.002003,0.015238,-0.004289,0.001231
573,-0.031353,0.007484,-0.047950,0.017633,0.023999,0.062482,0.006636,-0.003137,0.013271,-0.000315,...,-0.038091,0.003040,0.005548,0.023769,-0.028980,0.005294,0.002676,0.045069,-0.010457,-0.016630
407,-0.014292,-0.058539,0.036092,-0.020122,-0.031474,-0.030419,-0.049890,0.003007,0.005996,0.050321,...,-0.032544,0.005598,-0.009416,0.007353,-0.013659,0.024790,0.007221,0.062372,-0.061168,-0.011244
297,-0.025686,-0.040592,-0.045948,0.002470,0.025108,0.017212,-0.068101,-0.011036,-0.086145,0.029043,...,0.003418,0.020622,-0.075456,0.070052,0.003829,0.008402,-0.066474,0.034543,-0.014876,0.052876


In [89]:
train_rets = train_tsla.values
test_rets = test_tsla.values

train_y = train_targ.values
test_y = test_targ.values

In [61]:
tsla_normalizer = tf.keras.layers.Normalization()
tsla_normalizer.adapt(train_rets)

In [86]:
tsla_model = tf.keras.models.Sequential([
    tf.keras.layers.Input((len(train_rets[0]), )),
    tf.keras.layers.Normalization(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(2)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

tsla_model.compile(
    optimizer='adam',
    loss = loss_fn,
    metrics = ['accuracy']
)


In [95]:
tsla_model.fit(train_rets, train_y, epochs=400)

Epoch 1/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9763 - loss: 0.1056 
Epoch 2/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9562 - loss: 0.1460 
Epoch 3/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9682 - loss: 0.1501 
Epoch 4/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9691 - loss: 0.1368 
Epoch 5/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9781 - loss: 0.1135 
Epoch 6/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9765 - loss: 0.1118 
Epoch 7/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9815 - loss: 0.1074 
Epoch 8/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9632 - loss: 0.1383 
Epoch 9/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9535 - loss: 0.1401 
Epoch 10/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9620 - loss: 0.1370 
Epoch 11/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9428 - loss: 0.1595 
Epoch 12/400
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

KeyboardInterrupt: 

In [97]:
tsla_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_5 (Normalization) │ (None, 25)             │            51 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,435 (21.24 KB)

 Trainable params: 1,794 (7.01 KB)

 Non-trainable params: 51 (208.00 B)

 Optimizer params: 3,590 (14.03 KB)

In [96]:
tsla_model.evaluate(test_targ, test_y)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7245 - loss: 0.2760 


[0.3324279189109802, 0.6855670213699341]

In [93]:
test_targ.describe()

,Did I Hit?
count,194.000000
mean,0.685567
std,0.465491
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [88]:
prob_tsla = tf.keras.Sequential([
  tsla_model,
  tf.keras.layers.Softmax()
])